In [1]:
import glob
import sys 
import os
import xarray as xr
from dask.distributed import Client

In [2]:
c = Client()
c

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 7
Total threads: 28,Total memory: 0.98 TiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:41243,Workers: 7
Dashboard: /proxy/8787/status,Total threads: 28
Started: Just now,Total memory: 0.98 TiB
Comm: tcp://127.0.0.1:38167,Total threads: 4
Dashboard: /proxy/45019/status,Memory: 143.87 GiB
Nanny: tcp://127.0.0.1:37347,


In [3]:
#Data path
data_path="/g/data/w97/Shared_data/AWAP_3h_v1/"

variables=['Qair', "Tair"]

In [4]:
for v in variables:
    
    print(v)
    
    #Find files 

    #1970s
    files_70s=glob.glob(str(data_path + '/' + v + '/*197*.nc'))
    all_files=files_70s

    #1980s
    files_80s=glob.glob(str(data_path + '/' + v + '/*198*.nc'))
    all_files.extend(files_80s)

    #1990s
    files_90s=glob.glob(str(data_path + '/' + v + '/*199*.nc'))
    all_files.extend(files_90s)



    #Calculate mean Qair and temperature during 1970-1999

    #Open data handle
    ds = xr.open_mfdataset(all_files, chunks={'time': '400mb'}, parallel=True)

    #Get data (not exatcly sure what this does)
    #got the code from here: https://stackoverfstlow.com/questions/55997826/averaging-2-decades-of-data-on-6-hourly-timeep-using-netcdf-data-and-python
    ds['hourofyear'] = xr.DataArray(ds.indexes['time'].strftime('%m-%d %H'), coords=ds.time.coords)
    climatology = ds.groupby('hourofyear').mean('time')



    ### Write to file ###
    #outdir=str(data_path + "/../" + v + "_climatology/")
    #os.system("mkdir -p " + outdir)

    outfile=str("/scratch/public/sg7549/AWAP." + v + ".3hr.climatology_1970_1999.nc")

    #Rename data variable
    #climatology.name = v
    
    #5pm
    #Write to file
    #ncdat.load().to_netcdf('test_faster.nc')
    
    climatology.to_netcdf(outfile, format='NETCDF4', 
                   encoding={v:{
                             'shuffle':True,
                             'chunksizes':[12, 681, 40],
                             'zlib':True,
                             'complevel':5}
                             })

Qair
Tair


### Trying to use resample instead as it should be more efficient

This isn't working yet.

In [9]:
ds_sort = ds.sortby('time')

In [ ]:
#ds_sort = ds.sortby('time')

ds_resampled = ds_sort.resample(time='1H').mean()

# Calculate climatology
climatology = ds_resampled.groupby('time.hour').mean('time')